# The Basics

## Authentication

You can authenticate to the device using a password or ssh keys. For automated scripts it is recommended to use SSH keys. For interactive scripts you can prompt the user for the parameters using the input() function and storing the results in a variable.

    hostname = input("Device Hostname:")
    username = input("Username:")
    password = input("Password:")
    
Since ` input() ` will show the password in cleartext you can use the getpass method:

    import getpass
    password = getpass.getpass()
    
The input from the example below is used in all subsequent sections.

*Hint: If you need to change these values just re-run this cell.*



In [ ]:
import getpass
hostname = input("Device Hostname:")
username = input("Username:")
password = getpass.getpass()

These variables can be used anywhere by putting calling them by name.

In [ ]:
print(hostname)
print(username)
print(password)

## Connecting to a device

It is possible to connect to a device via Serial, Telnet or SSH.

Before you can use PyEZ to connect to the device via SSH you must enable netconf with the following commands:

    set system services ssh
    set system services netconf ssh

Next you can initiate the connection to the device. Note that when SSH is used PyEZ automatically retrieves the device facts. Edit the code below:

In [ ]:
from jnpr.junos import Device
from pprint import pprint

dev = Device(host=hostname, user=username, password=password, port='22')
dev.open()

pprint (dev.facts)

dev.close()

## Sample Output

    {'2RE': False,
     'HOME': '/cf/root',
     'RE0': {'last_reboot_reason': '0x10:misc hardware reason ',
             'model': 'VSRX RE',
             'status': 'Testing',
             'up_time': '23 hours, 36 minutes, 29 seconds'},
     'domain': None,
     'fqdn': 'vsrx2',
     'hostname': 'vsrx2',
     'ifd_style': 'CLASSIC',
     'model': 'VSRX',
     'personality': 'UNKNOWN',
     'serialnumber': '036aa3338cf5',
     'srx_cluster': False,
     'switch_style': 'NONE',
     'vc_capable': False,
     'version': '15.1X49-D50.3',
     'version_info': junos.version_info(major=(15, 1), type=X, minor=(49, 'D', 50), build=3)}

## Configuring a device

There are two methods to configuring the device, structured and unstructured.

### Structured Configuration Changes

Structured configuration changes are useful when the user does not know configuration stanzas or how to configure a Junos device. It abstracts the configuration statements from the user and allows changes to be made through tables and views.

### Unstructured Configuration Changes

Unstructured configuration changes allow the user to make changes via "normal" configuration commands using XML, Junos set commands or JSON. These changes are performed using the ` jnpr.junos.utils.config.Config ` utility. 

After connecting to the device you create a `Config` object and associate it to the device:

    from jnpr.junos import Device
    from jnpr.junos.utils.config import Config

    dev = Device(host=hostname, user=username, password=password, port='22')
    cu = Config(dev)
    
The basic process for making changes is to:

1. Lock the configuarion using ` lock() `
2. Use `load()`, `rollback()` or `rescue()` to make changes
3. Commit the configuration using `commit()`
4. Unlock the configuration using `unlock()`


### Unstructured Example

*Note, I had to adjust the timeout on commit() to 120 seconds since my compute node is slow and the vSRX was taking >30 seconds (the default timeout) to commit.*

In [ ]:
from jnpr.junos import Device
from jnpr.junos.utils.config import Config

dev = Device(host=hostname, user=username, password=password, port='22')

dev.open()

with Config(dev, mode='private') as cu:  
    cu.load('set system services netconf traceoptions file test.log', format='set')
    cu.pdiff()
    cu.commit(timeout=120)
    
    
dev.close()

### Rollback Example

In [ ]:
from jnpr.junos import Device
from jnpr.junos.utils.config import Config

dev = Device(host=hostname, user=username, password=password, port='22')

dev.open()

with Config(dev, mode='private') as cu:  
    cu.rollback(1)
    cu.pdiff()
    cu.commit(timeout=120)
    
dev.close()